# About

Sentiment analysis is contextual mining of text which identifies and extracts subjective information in source material, and helping a business to understand the social sentiment of their brand, product or service while monitoring online conversations. Brands can use this data to measure the success of their products in an objective manner.

# Code

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [743]:
df=pd.read_csv('F:\\train_2kmZucJ.csv',encoding="utf-8")
df=df.drop('id',1)
test=pd.read_csv('F:\\test_oJQbWVk.csv',encoding="utf-8")
id=test['id']
test=test.drop('id',1)
df.head()

,label,tweet
0,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,0,Finally a transparant silicon case ^^ Thanks t...
2,0,We love this! Would you go? #talk #makememorie...
3,0,I'm wired I know I'm George I was made that wa...
4,1,What amazing service! Apple won't even talk to...


In [744]:
df['label'].value_counts()

0    5894
1    2026
Name: label, dtype: int64

In [745]:
df['tweet']=df['tweet'].str.replace('[^a-zA-Z]',' ')
test['tweet']=test['tweet'].str.replace('[^a-zA-Z]',' ')

In [746]:
df['tweet'] = df['tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
test['tweet'] = test['tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [747]:
tweet=df['tweet'].apply(lambda x:x.split())
tweet_test=test['tweet'].apply(lambda x:x.split())

In [748]:
from nltk.stem import WordNetLemmatizer
from nltk import sent_tokenize
le = WordNetLemmatizer()
tweet=tweet.apply(lambda x:[le.lemmatize(word) for word in x])

In [749]:
tweet_test=tweet_test.apply(lambda x:[le.lemmatize(word) for word in x])

In [750]:
for i in range(len(tweet)):
    tweet[i] =' '.join(tweet[i])    
df['tweet']=tweet

In [751]:
for i in range(len(tweet_test)):
    tweet_test[i] =' '.join(tweet_test[i])    
test['tweet']=tweet_test

In [752]:
import sys
reload(sys)
sys.setdefaultencoding('utf8')

In [753]:
import nltk
nltk.download('stopwords')

True

In [754]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer(stop_words='english',max_df=0.9,max_features=100)
tf.fit(df['tweet'])
df_tf=pd.DataFrame(tf.transform(df['tweet']).toarray(),columns=tf.get_feature_names()).add_prefix('tfidf_')
df=df.join(df_tf)
df=df.drop('tweet',1)

In [755]:
a=pd.DataFrame(tf.get_feature_names())
a.shape

(100, 1)

In [756]:
#from sklearn.feature_extraction.text import TfidfVectorizer
#tf=TfidfVectorizer(stop_words='english',max_df=0.9,max_features=100)
tf.fit(test['tweet'])
test_tf=pd.DataFrame(tf.transform(test['tweet']).toarray(),columns=tf.get_feature_names()).add_prefix('tfidf_')
test=test.join(df_tf)
test=test.drop('tweet',1)

In [757]:
df.shape

(7920, 101)

In [758]:
test.shape

(1953, 100)

In [759]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [760]:
x=df.drop('label',1)
y=df['label']

In [876]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42,stratify=y)

In [877]:
from xgboost import XGBClassifier
clf= XGBClassifier(n_estimators=50,subsample=0.8)
clf.fit(x_train,y_train)
clf.score(x_test,y_test)

0.8702020202020202

In [878]:
dt=DecisionTreeClassifier(max_depth=2)
dt.fit(x_train,y_train)
y_dt=dt.predict(x_test)
roc_auc_score(y_dt,y_test)

0.7713901477789702

In [879]:
from sklearn.ensemble import GradientBoostingClassifier
ada=GradientBoostingClassifier(n_estimators=1,subsample=0.8)
ada.fit(x_train,y_train)
ada.score(x_test,y_test)

0.7444444444444445

In [880]:
log=LogisticRegression(C=1)
log.fit(x_train,y_train)
log.score(x_test,y_test)

0.8722222222222222

In [881]:
y_test=clf.predict(test)

In [882]:
sub=pd.read_csv('F:\\sample_submission_LnhVWA4.csv')
sub.head()

,id,label
0,7921,0
1,7922,0
2,7923,0
3,7924,0
4,7925,0


In [883]:
sub['label']=y_test

In [884]:
sub['label'].value_counts()

0    1413
1     540
Name: label, dtype: int64

In [885]:
sub.to_csv('F:\\answer.csv',index=False)

In [886]:
sub

,id,label
0,7921,0
1,7922,0
2,7923,0
3,7924,0
4,7925,1
5,7926,1
6,7927,0
7,7928,0
8,7929,0
9,7930,0
